In [1]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Polygon, MultiPolygon

# Load the Shapefile
shapefile_path = '../Major Towns and Cities 2015/Major_Towns_and_Cities_Dec_2015_Boundaries_V2_2022_-2549004559791541639/TCITY_2015_EW_BGG_V2.shp'
mtc_gdf = gpd.read_file(shapefile_path)

shapefile_path_scot = '../Local Authority District 2011 Boundaries GB BFC/Local_Authority_Districts_December_2011_GB_BFC_2022_-7260447689016327025/Local_Authority_Districts_December_2011_GB_BFC.shp'
mtc_gdf_scot = gpd.read_file(shapefile_path_scot)

# Ensure the GeoDataFrame has the correct coordinate reference system (CRS)
mtc_gdf = mtc_gdf.to_crs(epsg=4326)
mtc_gdf_scot = mtc_gdf_scot.to_crs(epsg=4326)

In [2]:
mtc_gdf.head()

,TCITY15CD,TCITY15NM,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,J01000001,Barnsley,436375,407875,-1.452250,53.5662,b8f30b00-7b1b-40f9-aa0f-8336ec877f55,"POLYGON ((-1.51419 53.59639, -1.51344 53.59638..."
1,J01000002,Basildon,572525,189350,0.488409,51.5766,f1232e18-8e80-488e-a6e4-f7ce988993ba,"MULTIPOLYGON (((0.47013 51.59274, 0.47016 51.5..."
2,J01000003,Basingstoke,463725,152600,-1.087930,51.2690,e54ea899-ecd0-43a8-9249-3f42d005c8d7,"POLYGON ((-1.04627 51.29205, -1.04698 51.29205..."
3,J01000004,Bath,374800,164325,-2.363450,51.3774,db04cdf4-2fa4-4af4-9ffb-8e7f5afcaa0a,"MULTIPOLYGON (((-2.30755 51.39668, -2.30755 51..."
4,J01000005,Bedford,505925,250450,-0.453520,52.1422,9829a7ac-85ca-40fb-9bf1-a0f325648505,"MULTIPOLYGON (((-0.42637 52.15667, -0.42635 52..."


In [3]:
mtc_gdf_scot.head()

,lad11cd,lad11cdo,lad11nm,lad11nmw,GlobalID,geometry
0,E06000001,00EB,Hartlepool,None,bab24c73-2e01-4627-802b-edf35d74bef8,"MULTIPOLYGON (((-1.26844 54.72611, -1.26821 54..."
1,E06000002,00EC,Middlesbrough,None,8f377cd6-ddc5-4e7c-92f4-f6a146668819,"MULTIPOLYGON (((-1.24921 54.59069, -1.24802 54..."
2,E06000003,00EE,Redcar and Cleveland,None,685deceb-2ba2-49a5-bba4-78e113bdf17c,"MULTIPOLYGON (((-1.13756 54.64580, -1.13779 54..."
3,E06000004,00EF,Stockton-on-Tees,None,4c82c268-56e7-4615-a069-0c5f1b898baa,"MULTIPOLYGON (((-1.31767 54.64497, -1.31762 54..."
4,E06000005,00EH,Darlington,None,388c9e1a-8532-452b-bb24-6bfd5ed1570b,"POLYGON ((-1.63767 54.61713, -1.63766 54.61669..."


In [4]:
mtc_use = ["Cardiff", "Swansea", "Newport", "Birmingham", "Bradford", "Brighton and Hove",
            "Bristol", "Cambridge", "Carlisle", "Coventry", "Derby", "Doncaster", "Exeter",
            "Leeds", "Leicester", "Lincoln", "Liverpool", "Manchester", "Newcastle upon Tyne",
            "Norwich", "Nottingham", "Oxford",  "Portsmouth", "Preston", "Salford","London",
            "Sheffield", "Southampton", "Southend-on-Sea", "Stoke-on-Trent", "Sunderland", "York"]
mtc_use_scot = ["Glasgow City", "Edinburgh, City of"]
print(len(mtc_use))

32


In [5]:
import pandas as pd
mtc1 = mtc_gdf[mtc_gdf['TCITY15NM'].isin(mtc_use)][['TCITY15NM','TCITY15CD','geometry']]
mtc2 = mtc_gdf_scot[mtc_gdf_scot['lad11nm'].isin(mtc_use_scot)][['lad11nm','lad11cd','geometry']]
mtc2.columns = mtc1.columns
mtc = pd.concat([mtc1,mtc2], axis=0, ignore_index=True)
mtc['area'] = mtc['geometry'].to_crs(epsg=3857).area
mtc.head()

,TCITY15NM,TCITY15CD,geometry,area
0,Birmingham,J01000007,"MULTIPOLYGON (((-1.75530 52.52337, -1.75529 52...",6.168126e+08
1,Bradford,J01000013,"MULTIPOLYGON (((-1.69907 53.83922, -1.69907 53...",2.003226e+08
2,Brighton and Hove,J01000014,"POLYGON ((-0.15198 50.86362, -0.15197 50.86407...",7.897562e+07
3,Bristol,J01000015,"MULTIPOLYGON (((-2.67894 51.48189, -2.67894 51...",2.895024e+08
4,Cambridge,J01000019,"MULTIPOLYGON (((0.16076 52.23550, 0.16078 52.2...",1.000203e+08


In [6]:
mtc.shape

(34, 4)

In [7]:
# Define the function to get historical OSM data from ohsome API
def get_total_line_length(geometry, date="2009-01-01", filter_='type:way'):
    base_url = "https://api.ohsome.org/v1/elements/geometry"
    
    data = {
        "bpolys": geometry,
        "time": date,
        "filter": filter_
    }
    
    response = requests.post(base_url, data=data)
    
    if response.status_code == 200:
        features = response.json()['features']
        total_length = 0.0
        
        for feature in features:
            geometry = feature['geometry']
            if geometry['type'] in ['LineString', 'MultiLineString']:
                total_length += calculate_length(geometry['coordinates'])
        
        return total_length
    else:
        print(f"Error fetching data: {response.status_code}")
        print(response.json())
        return None
    
def calculate_length(coordinates):
    """Calculate the length of a LineString or MultiLineString based on coordinates."""
    total_length = 0.0
    
    if isinstance(coordinates[0][0], list):  # It's a MultiLineString
        for line in coordinates:
            total_length += calculate_length(line)
    else:  # It's a LineString
        for i in range(len(coordinates) - 1):
            start_point = (coordinates[i][1], coordinates[i][0])
            end_point = (coordinates[i+1][1], coordinates[i+1][0])
            total_length += geodesic(start_point, end_point).meters
    
    return total_length

def polygon_to_coord_list(polygon):
    if isinstance(polygon, Polygon):
        polygon = make_valid(polygon)
        coords = list(polygon.exterior.coords)
        coord_list = ",".join([f"{x},{y}" for x, y in coords])
        return [coord_list]
    elif isinstance(polygon, MultiPolygon):
        coord_list = []
        for poly in polygon.geoms:
            poly = make_valid(poly)
            coords = list(poly.exterior.coords)
            coord_list.append(",".join([f"{x},{y}" for x, y in coords]))
        return coord_list
    else:
        raise TypeError("Unsupported geometry type")

In [8]:
filters = [
# driving
"type:way and (highway in (motorway,trunk,primary,secondary,tertiary,unclassified,residential,motorway_link,trunk_link,primary_link,secondary_link,tertiary_link))",
# cycling
"type:way and (cycleway=*)",
# cycling
"type:way and (highway=cycleway)",
# walking
"type:way and (sidewalk=*)",
# walking
"type:way and (highway=footway)",
# walking
"type:way and (footway=*)"
]

In [9]:
# initiate an empty columns for result recording
import pandas as pd

var_names = ['driving','cycling_cycleway','cycling_highway','walking_sidewalk','walking_highway','walking_footway']

for var in var_names:
    mtc[var] = [pd.NA] * mtc.shape[0]


In [10]:
mtc.head()

,TCITY15NM,TCITY15CD,geometry,area,driving,cycling_cycleway,cycling_highway,walking_sidewalk,walking_highway,walking_footway
0,Birmingham,J01000007,"MULTIPOLYGON (((-1.75530 52.52337, -1.75529 52...",6.168126e+08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,Bradford,J01000013,"MULTIPOLYGON (((-1.69907 53.83922, -1.69907 53...",2.003226e+08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Brighton and Hove,J01000014,"POLYGON ((-0.15198 50.86362, -0.15197 50.86407...",7.897562e+07,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Bristol,J01000015,"MULTIPOLYGON (((-2.67894 51.48189, -2.67894 51...",2.895024e+08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Cambridge,J01000019,"MULTIPOLYGON (((0.16076 52.23550, 0.16078 52.2...",1.000203e+08,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [11]:
mtc.shape

(34, 10)

In [12]:
for y in ['2019','2014','2012','2010']:
    mtc.to_csv('LAD_streetnetwork_'+y+'.csv')

In [13]:
import requests
from geopy.distance import geodesic
import re
from shapely.validation import make_valid
from tqdm import tqdm
from shapely import wkt

for y in ['2019','2014','2012','2010']:
    london = pd.read_csv('LAD_streetnetwork_'+y+'.csv')
    london['geometry'] = london['geometry'].apply(wkt.loads)
    for i, row in tqdm(london.iterrows(), total=london.shape[0]):
        if row.isna().any() or (-1 in row.values):
            geometry_list = polygon_to_coord_list(row['geometry'])
            for j in range(len(filters)):
                if pd.isna(row[var_names[j]]) or (row[var_names[j]] == -1):
                    try:
                        total_length = 0
                        for geometry in geometry_list:
                            length = get_total_line_length(geometry, y+'-01-01', filters[j])
                            total_length += length
                        london.at[i, var_names[j]] = total_length
                    except Exception as e:
                        print(f"Error fetching POI data for LSOA {row['LSOA01CD']}: {e}")
                        london.at[i, var_names[j]] = -1

            # Save the updated DataFrame to the CSV file
            if i % 3 == 0:
                london.to_csv('LAD_streetnetwork_'+y+'.csv', index=False)
    london.to_csv('LAD_streetnetwork_'+y+'.csv', index=False)

100%|██████████| 34/34 [10:29<00:00, 18.50s/it]


In [14]:
stats = []
for y in ['2010','2012','2014','2019']:
    mtc = pd.read_csv('LAD_streetnetwork_'+y+'.csv')
    year_stats = []
    for v in var_names:
        year_stats.append(mtc[v].sum())
    stats.append(year_stats)
stats = pd.DataFrame(stats)
stats.columns = var_names
    

In [15]:
stats.index = ['2010','2012','2014','2019']

In [16]:
stats

# results should delete:
# amenity_biergarten, shop_brewing_supplies, shop_cannabis

,driving,cycling_cycleway,cycling_highway,walking_sidewalk,walking_highway,walking_footway
2010,3.865244e+07,5.691761e+05,1.061138e+06,0.000000e+00,4.291475e+06,2838.107161
2012,4.329290e+07,8.248368e+05,1.292198e+06,2.697634e+04,6.461786e+06,138836.717077
2014,4.410720e+07,1.072473e+06,1.539248e+06,2.165052e+06,8.574555e+06,200682.206659
2019,4.460756e+07,1.244668e+06,2.305088e+06,4.182100e+06,1.153347e+07,567865.665253


In [18]:
# backup function

# import json
# from shapely.geometry import Polygon, MultiPolygon

# def polygon_to_geojson_feature_collection(polygon, region_id='Region'):
#     features = []

#     def format_coordinates(coords):
#         return [[list(coord) for coord in coords]]

#     if isinstance(polygon, Polygon):
#         coordinates = format_coordinates(polygon.exterior.coords)
#         feature = {
#             "type": "Feature",
#             "properties": {"id": f"{region_id} 1"},
#             "geometry": {
#                 "type": "Polygon",
#                 "coordinates": coordinates
#             }
#         }
#         features.append(feature)

#     elif isinstance(polygon, MultiPolygon):
#         for idx, poly in enumerate(polygon.geoms):
#             coordinates = format_coordinates(poly.exterior.coords)
#             feature = {
#                 "type": "Feature",
#                 "properties": {"id": f"{region_id} {idx + 1}"},
#                 "geometry": {
#                     "type": "Polygon",
#                     "coordinates": coordinates
#                 }
#             }
#             features.append(feature)
#     else:
#         raise TypeError("Unsupported geometry type")

#     feature_collection = {
#         "type": "FeatureCollection",
#         "features": features
#     }
#     return feature_collection


{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'id': 'Region 1'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-1.5704498471484643, 54.923952175382496], [-1.5704497077420054, 54.92395207963483], [-1.5701740091293421, 54.92406343507078], [-1.570067292381269, 54.92410507449073], [-1.5699574872283233, 54.92414791875118], [-1.5698547810752077, 54.92418799326264], [-1.569747889753728, 54.924229700081426], [-1.5697289359590474, 54.92423709530048], [-1.5696262192179127, 54.92428016742882], [-1.569531771414466, 54.92431977124733], [-1.5694002422589792, 54.924374924008276], [-1.5692680481964985, 54.92443035554523], [-1.5690005287472784, 54.92455891469998], [-1.568879389802446, 54.9246171303187], [-1.5688792645069296, 54.92461702832883], [-1.5688746180357085, 54.92461321871992], [-1.5688275856568914, 54.924574662304586], [-1.5682360406280935, 54.92408970825292], [-1.5679862770907975, 54.92390418146634], [-1.567864526165285, 54.92381374343603], [-1.567676365597